# Analyzing DOTA 2's TI8

## Purpose
My purpose in creating this project was to practice my newly acquired proficiency in the Python language, along with a few very common data science packages for the language. I chose to complete my first data analysis for a video game I have been in love with for the past 5 years, DOTA 2. DOTA 2 is categorized as a MOBA and is played on a computer. The game has over 10 million monthly players across the globe and has garnered a large viewership for its competitive events. Every August *The International* (DOTA 2's World Championships) is held with the 18 best teams in the world competing for a large prize pool. For the past five internationals the prize pool of the tournament has been crowd-funded, meaning that the player base contributes to the prize pool by purchasing in-game cosmetic items. The prize pool for this years International, TI8, was over $25 million.

Throughout this analysis I performed get requests on the [OpenDota API](https://docs.opendota.com/) to access a multitude of data points related to the game. I performed this analysis across all 195 games of TI8, including both the tournament's group stage and main event LAN tournament, however I did not look at data from the qualifiers. This write-up assumes a surface level understanding of the game. If you would like to learn a little bit more about the game and have two minutes to spend, check out [This is DOTA](https://www.youtube.com/watch?v=Cp8neRiF9-k) published by the game's developer Valve Software. I hope you enjoy

I began by importing all the necessary packages for this data analysis project. 
In this first cell my personal API key for OpenDota's API is saved to a variable called api-key. 
This API key will be useful if a large number of calls to the OpenDota API are required in quick succession (users without a key are only allowed to make 60 calls per minute).

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os.path
from pathlib2 import Path
import pickle
import time

api_key = '?api_key=082df0c9-30cb-4351-8052-b3c709f22ba7'

Next it was helpful to create a list consisting of the match ID numbers for every game played at The International 8. 
This list was saved to a pickled file in order to save it for future use without needing to access the API repeatedly.
I also saved a pickled file that represents all the team matchups that occurred during the tournament. This was important to producing team-based statistics as the OpenDota API doesn't provide a team name and ID from a *matches* get request, only from the less useful *pro matches* call.
Note: This cell does not unpack the pickled file if it does actually exist, only creates it if it does not.

In [2]:
file = Path('match_ids.pickle')    
if not file.exists() :
    cont = True
    match_ids = []
    matchups = {}
    less_than_match = ''
    while cont:
        url = 'https://api.opendota.com/api/proMatches' + less_than_match

        r = requests.get(url)
        json_data = r.json()

        for i in range(len(json_data)) :
            if json_data[i].get('league_name') == 'The International 2018' :
                match_ids.append(json_data[i].get('match_id'))
                matchups[json_data[i].get('match_id')] = [json_data[i].get('dire_name'), json_data[i].get('radiant_name')]
            elif len(match_ids) > 0 :
                cont = False
                break
        if len(match_ids) == 0 :
            less_than_match = '?less_than_match_id=' + str(json_data[i].get('match_id'))
        else :
            less_than_match = '?less_than_match_id=' + str(match_ids[len(match_ids) - 1])
    with open('match_ids.pickle', 'wb') as outfile:  
        pickle.dump(match_ids, outfile)
    with open('matchups.pickle', 'wb') as outfile:  
        pickle.dump(matchups, outfile)

Next I created a pickled file that consists of an API call that returns a general dataset of all 116 heroes that exist in the game.

In [3]:
file = Path('hero.pickle')
if not file.exists() :
    url = 'https://api.opendota.com/api/heroes'
    r = requests.get(url)
    hero_json = r.json()
    with open('hero.pickle', 'wb') as outfile:  
        pickle.dump(hero_json, outfile)

Using the 'hero.pickle' file, I created a dictionary where each hero represents a key and their values are an empty NumPy array.
This array will be used later to keep a running tally of stats for that hero throughout the tournament with each index of the arrray representing a different data point (kills, assists, etc.).

In [4]:
with open('hero.pickle', 'rb') as json_file:  
    hero_json = pickle.load(json_file)

index = []
hero_ids = []

for i in range(len(hero_json)) :
    hero = hero_json[i]
    index.append(hero['localized_name'])
    hero_ids.append(hero['id'])

hero_stats = {hero_id: np.zeros(10) for hero_id in hero_ids}

Function *matchParser* takes the JSON data for a single match, accessed via its unique match ID number, and populates the dictionaries that correspond to statistics collected by hero, player, or team.

In [5]:
def matchParser(json_data, hero_stats, player_stats, player_hero_pool, matchups, team_stats, team_hero_pool) :
    draft_timings_json = json_data['draft_timings']
    players_json = json_data['players']
    for i in range(10) : # for each player/hero in the game
        player = players_json[i]
        hero_id = player['hero_id']
        player_id = player['name']
        side = int(player['isRadiant'])
        match_id = player['match_id']
        team = matchups[match_id][side]
        
        if not team_stats.has_key(team) :
            team_stats[team] = np.zeros(8)
            team_stats[team][3] = 100 # because index 3 is 'fewest deaths' and a default value of 0 will always be the lowest possible
            team_stats[team][7] = 5000 # same reason as above for the 'shortest game' value
            
        curr_hero_stats_array = np.zeros(10) # creates a NumPy array that will serve as a tally of statstics for the current game
        curr_player_stats_array = np.zeros(11) # NumPy array that serves the same purpose as above for stats by player
        if (i == 5 or i == 0) :
            curr_team_stats_array = np.zeros(8)
            curr_team_kills = 0
            curr_team_assists = 0
            curr_team_deaths = 0

        curr_hero_stats_array[0] = player.get('win')
        curr_hero_stats_array[1] = 1 # used to keep a count of the number of games that the hero was played during the tournament
        curr_hero_stats_array[3] = player['kills']
        curr_hero_stats_array[4] = player['assists']
        curr_hero_stats_array[7] = player['last_hits']
        curr_hero_stats_array[8] = player['xp_per_min']
        curr_hero_stats_array[9] = player['deaths']
        
        curr_player_stats_array[0] = 1 # also keeps count of the number of games the player has played during TI8
        curr_player_stats_array[1] = player['kills']
        curr_player_stats_array[3] = player['deaths']
        curr_player_stats_array[4] = player['assists']
        curr_player_stats_array[6] = player['last_hits']
        curr_player_stats_array[8] = player['gold_per_min']
        
        curr_team_stats_array[0] = 1
        curr_team_kills += player['kills']
        curr_team_assists += player['assists']
        curr_team_deaths += player['deaths']
        
        hero_stats_array = hero_stats[hero_id]
        hero_stats[hero_id] = np.add(curr_hero_stats_array, hero_stats_array) # adds stats for the current game together with the running total in hero_stats_array

        # checks to see if the current number of kills is greater than the hero's maximum kills and switches the max kills value if necessary
        most_kills = hero_stats_array[5] 
        curr_kills = curr_hero_stats_array[3]
        if most_kills < curr_kills :
            hero_stats[hero_id][5] = curr_kills
            
        # the logic above is repeated for last hits
        most_last_hits = hero_stats_array[6]
        curr_last_hits = curr_hero_stats_array[7]
        if most_last_hits < curr_last_hits :
            hero_stats[hero_id][6] = curr_last_hits
            
        if not player_stats.has_key(player_id) :
            player_stats[player_id] = np.zeros(11)
        
        player_stats_array = player_stats[player_id]
        player_stats[player_id] = np.add(curr_player_stats_array, player_stats_array)
            
        most_kills = player_stats_array[2]
        curr_kills = curr_player_stats_array[1]
        if most_kills < curr_kills :
            player_stats[player_id][2] = curr_kills
        
        most_assists = player_stats_array[5]
        curr_assists = curr_player_stats_array[4]
        if most_assists < curr_assists :
            player_stats[player_id][5] = curr_assists
        
        most_last_hits = player_stats_array[7]
        curr_last_hits = curr_player_stats_array[6]
        if most_last_hits < curr_last_hits :
            player_stats[player_id][7] = curr_last_hits
        
        highest_gpm = player_stats_array[9]
        curr_gpm = curr_player_stats_array[8]
        if highest_gpm < curr_gpm :
            player_stats[player_id][9] = curr_gpm
            
        if (i == 4 or i == 9) :
            team_stats_array = team_stats[team]
            curr_team_stats_array[2] = curr_team_kills
            team_stats[team] = np.add(curr_team_stats_array, team_stats_array)

            most_kills = team_stats_array[1]
            curr_kills = curr_team_kills
            if most_kills < curr_kills :
                team_stats[team][1] = curr_kills

            most_assists = team_stats_array[4]
            curr_assists = curr_team_assists
            if most_assists < curr_assists :
                team_stats[team][4] = curr_assists

            least_deaths = team_stats_array[3]
            curr_deaths = curr_team_deaths
            if least_deaths > curr_deaths :
                team_stats[team][3] = curr_deaths
                
            if(player['win'] == 1) :
                longest_game = team_stats_array[6]
                shortest_game = team_stats_array[7]
                curr_length = json_data['duration']
                if longest_game < curr_length :
                    team_stats[team][6] = curr_length
                if shortest_game > curr_length :
                    team_stats[team][7] = curr_length
            
        if not player_hero_pool.has_key(player_id) : # checks if a given player does not already have an entry in player_hero_pool
            player_hero_pool[player_id] = set([]) # if they don't have an entry, an empty set is created for that player
        player_hero_pool[player_id].add(hero_id) # the hero ID of the current hero is added to that player's 'hero pool'
        
        if not team_hero_pool.has_key(team) : # checks if a given team does not already have an entry in team_hero_pool
            team_hero_pool[team] = set([]) # if they don't have an entry, an empty set is created for that team
        team_hero_pool[team].add(hero_id) # the hero ID of the current hero is added to that team's 'hero pool' set
            
    for j in range(22) : # for
        curr_phase = draft_timings_json[j]
        if curr_phase['pick'] == False :
            hero_stats[curr_phase['hero_id']][2] += 1
    return hero_stats, player_stats, player_hero_pool, team_hero_pool, team_stats

Since the NumPy array tied to each hero in the dictionary only contains a running total of metrics like kills throughout the tournament, *convertHeroStats* translates these total values to averages per game to standardize it against other heroes.

In [6]:
def convertHeroStats(hero_stats) :
    for hero in hero_stats.keys() :
        number_of_games = hero_stats[hero][1]
        if number_of_games > 0 :
            hero_stats[hero][0] /= number_of_games
            hero_stats[hero][3] /= number_of_games
            hero_stats[hero][4] /= number_of_games
            hero_stats[hero][7] /= number_of_games
            hero_stats[hero][8] /= number_of_games
            hero_stats[hero][9] /= number_of_games
    return hero_stats

*convertPlayerStats* acts similarly to the function above, just in the context of the player stats dictionary. It also gives the player a value for the number of unique heroes they played by finding the length of their "hero pool" set that stores the heroID for every hero they played during the tournament.

In [7]:
def convertPlayerStats(player_stats, player_hero_pool) :
    for player in player_stats.keys() :
        number_of_games = player_stats[player][0]
        if number_of_games > 0 :
            player_stats[player][1] /= number_of_games
            player_stats[player][3] /= number_of_games
            player_stats[player][4] /= number_of_games
            player_stats[player][6] /= number_of_games
            player_stats[player][8] /= number_of_games
            player_stats[player][10] = len(player_hero_pool[player])
    return player_stats

As before, *convertTeamStats* works just like the previous two functions, but for the team statistics dictionary.

In [8]:
def convertTeamStats(team_stats, team_hero_pool) :
    for team in team_stats.keys() :
        number_of_games = team_stats[team][0] 
        if number_of_games > 0 :
            team_stats[team][2] /= number_of_games
            team_stats[team][5] = len(team_hero_pool[team])
    return team_stats

As I mentioned before, I have yet to open the .pickle files created previously. I started by loading the list of the tournament's match_id's into a list, along with loading the matchups dictionary from its file. Then, for each of TI8's 195 games, I made a get request to the API for a specific match ID, saved the response as a JSON, dumped that JSON into a .pickle for future access, and finally passed the JSON onto the *matchParser* function to populate the three main data dictionaries.

In [9]:
player_stats = {}
player_hero_pool = {}
team_hero_pool = {}
team_stats = {}
with open('match_ids.pickle', 'rb') as json_file:  
    match_ids = pickle.load(json_file)
with open('matchups.pickle', 'rb') as json_file:  
    matchups = pickle.load(json_file)
for j in range(len(match_ids)) :
    path = str(match_ids[j]) + '.pickle'
    file = Path(path)
    if not file.exists() :
        url = 'https://api.opendota.com/api/matches/' + str(match_ids[j])
        r = requests.get(url)
        json_data = r.json()
        with open(path, 'wb') as outfile:  
            pickle.dump(json_data, outfile)
    else :
        with open(path, 'rb') as data:
            json_data = pickle.load(data)
    hero_stats, player_stats, player_hero_pool, team_hero_pool, team_stats = matchParser(json_data, hero_stats, player_stats, player_hero_pool, matchups, team_stats, team_hero_pool)

Once all the games have been parsed, I have three dictionaries that can each be converted into pandas DataFrames to represent the data in a more visually appealing manner. I converted the heroes directory to a DataFrame and defined the DataFrame's index and columns to increase clarity (instead of displaying heroes by their ID).

In [10]:
columns = ['Winrate', 'Games Picked', 'Games Banned', 'Kill Average',
        'Assist Average', 'Most Kills', 'Most Last Hits',
        'Average Last Hits', 'Average XPM', 'Death Average']
hero_stats = convertHeroStats(hero_stats)
heroes = pd.DataFrame.from_dict(hero_stats, orient='index', columns=columns)
heroes.index = index
heroes.head()

,Winrate,Games Picked,Games Banned,Kill Average,Assist Average,Most Kills,Most Last Hits,Average Last Hits,Average XPM,Death Average
Anti-Mage,0.000000,1.0,2.0,4.000000,4.000000,4.0,331.0,331.000000,681.000000,5.000000
Axe,0.666667,3.0,3.0,13.000000,9.333333,18.0,298.0,211.666667,519.333333,4.333333
Bane,0.555556,36.0,19.0,2.805556,13.722222,8.0,107.0,29.305556,348.777778,7.805556
Bloodseeker,0.483871,31.0,47.0,8.774194,11.806452,22.0,561.0,268.096774,535.387097,5.290323
Crystal Maiden,0.500000,52.0,40.0,2.384615,13.038462,9.0,171.0,61.250000,359.307692,7.288462


For creating the player statistics DataFrame I followed the same procedure as above, although the index did not need to be defined because the keys of the dictionary were already player names.

In [11]:
columns2 = ['Games Played', 'Kill Average', 'Most Kills', 'Death Average',
            'Assist Average', 'Most Assists', 'Last Hit Average',
            'Most Last Hits', 'GPM Average', 'Highest GPM', 'Heroes Played']
player_stats = convertPlayerStats(player_stats, player_hero_pool)
players = pd.DataFrame.from_dict(player_stats, orient='index', columns=columns2)
players.head()

,Games Played,Kill Average,Most Kills,Death Average,Assist Average,Most Assists,Last Hit Average,Most Last Hits,GPM Average,Highest GPM,Heroes Played
raining,18.0,4.333333,13.0,5.611111,10.166667,24.0,181.055556,356.0,384.611111,587.0,13.0
EternaLEnVy,17.0,4.705882,17.0,5.588235,8.294118,16.0,295.352941,588.0,476.235294,594.0,11.0
RAMZES666,25.0,9.520000,15.0,4.080000,11.280000,29.0,279.920000,538.0,558.440000,741.0,13.0
xNova,30.0,2.866667,9.0,7.066667,14.700000,28.0,54.266667,113.0,285.266667,412.0,13.0
YapzOr,26.0,4.615385,11.0,4.692308,16.461538,32.0,80.115385,224.0,306.961538,437.0,11.0


Last I created the DataFrame for team statistics as I did twice before. This process included a couple more manipulations because the *Longest Game* and *Shortest Game* statistics were referenced in number of seconds because all indexes of a NumPy array are required to have the same data type (in this case float). I removed those columns from the DataFrame once it was created and then converted them to a timedelta64 datatype and added them back to the DataFrame. There was also a weird issue where *Optic Gaming* was being referenced as an empty string from the API, so I manually changed it.

In [12]:
columns3 = ['Games Played', 'Most Kills', 'Kill Average', 'Fewest Deaths', 'Most Assists', 
           'Heroes Played', 'Longest Game', 'Shortest Game']
team_stats = convertTeamStats(team_stats, team_hero_pool)
teams = pd.DataFrame.from_dict(team_stats, orient='index', columns=columns3)
longest_game = teams.iloc[:, 6:7].values # because NumPy arrays must contain all the same value types, the last two columns are removed
shortest_game = teams.iloc[:, 7:8].values #
teams = teams.iloc[:, 0:6]
longest_game_time = longest_game.astype('timedelta64[s]').tolist()
shortest_game_time = shortest_game.astype('timedelta64[s]').tolist()
teams['Longest Game'] = longest_game_time
teams['Shortest Game'] = shortest_game_time
index = teams.index.values
index[0] = 'Optic Gaming'
teams.index = index
teams.head()

,Games Played,Most Kills,Kill Average,Fewest Deaths,Most Assists,Heroes Played,Longest Game,Shortest Game
Optic Gaming,24.0,51.0,25.083333,9.0,117.0,52.0,[0:47:28],[0:29:06]
Newbee,19.0,48.0,28.789474,13.0,116.0,39.0,[1:19:08],[0:32:10]
Team Secret,26.0,47.0,28.846154,2.0,107.0,58.0,[0:55:20],[0:18:24]
Virtus.pro,25.0,60.0,31.160000,10.0,115.0,46.0,[0:53:47],[0:22:35]
OG,29.0,52.0,30.000000,6.0,120.0,47.0,[1:05:21],[0:16:59]


## Analyzing Predictions

Now that DataFrames have been created for all the heroes, players, and teams at TI8, we can now look at the results of various predictions post-tournament. 

### General tournament predictions

Total number of heroes picked: 110

In [13]:
heroes_picked = heroes[heroes['Games Picked'] > 0]
heroes_picked.shape

(110, 10)

Total number of heroes banned: 96

In [14]:
heroes_banned = heroes[heroes['Games Banned'] > 0]
heroes_banned.shape

(96, 10)

### Predictions related to specific heroes

First is the most picked hero at the tournament: Vengeful Spirit

In [15]:
heroes.sort_values(by='Games Picked', ascending=False, inplace=True)
heroes.iloc[:,1:2].head()

,Games Picked
Vengeful Spirit,89.0
Tiny,78.0
Mirana,74.0
Wraith King,66.0
Dark Willow,66.0


Next I found the most banned hero: Enchantress

In [16]:
heroes.sort_values(by='Games Banned', ascending=False, inplace=True)
heroes.iloc[:,2:3].head()

,Games Banned
Enchantress,150.0
Io,140.0
Weaver,117.0
Necrophos,114.0
Silencer,107.0


Hero with the most kills in a game: Tiny

In [17]:
heroes.sort_values(by='Most Kills', ascending=False, inplace=True)
heroes.iloc[:,5:6].head()

,Most Kills
Tiny,31.0
Clinkz,24.0
Bloodseeker,22.0
Invoker,22.0
Weaver,21.0


Hero with the most last hits in a game: Tinker

In [18]:
heroes.sort_values(by='Most Last Hits', ascending=False, inplace=True)
heroes.iloc[:,6:7].head()

,Most Last Hits
Tinker,864.0
Timbersaw,758.0
Terrorblade,731.0
Arc Warden,686.0
Lina,672.0


In [19]:
heroes = heroes[heroes['Games Picked'] >= 5]

Hero with the highest winrate: Venomancer

In [20]:
heroes.sort_values(by='Winrate', ascending=False, inplace=True)
heroes.iloc[:,0:1].head()

,Winrate
Venomancer,0.692308
Drow Ranger,0.689655
Arc Warden,0.666667
Invoker,0.666667
Broodmother,0.666667


Hero with the highest kill average: Timbersaw

In [21]:
heroes.sort_values(by='Kill Average', ascending=False, inplace=True)
heroes.iloc[:,3:4].head()

,Kill Average
Timbersaw,10.000000
Gyrocopter,9.588235
Invoker,9.458333
Broodmother,9.388889
Clinkz,9.254902


Hero with the highest assist average: Tusk

In [22]:
heroes.sort_values(by='Assist Average', ascending=False, inplace=True)
heroes.iloc[:,4:5].head()

,Assist Average
Tusk,17.466667
Nyx Assassin,17.050000
Vengeful Spirit,16.988764
Phoenix,16.821429
Earth Spirit,16.700000


Hero with the lowest death average: Medusa

In [23]:
heroes.sort_values(by='Death Average', inplace=True)
heroes.iloc[:,9:10].head()

,Death Average
Medusa,2.666667
Broodmother,3.166667
Lycan,3.250000
Faceless Void,3.375000
Templar Assassin,3.615385


Hero with the highest last hit average: Shadow Fiend

In [24]:
heroes.sort_values(by='Average Last Hits', ascending=False, inplace=True)
heroes.iloc[:,7:8].head()

,Average Last Hits
Shadow Fiend,475.833333
Terrorblade,437.352941
Medusa,427.833333
Alchemist,397.500000
Timbersaw,394.800000


Hero with the highest XPM average: Timbersaw

In [25]:
heroes.sort_values(by='Average XPM', ascending=False, inplace=True)
heroes.iloc[:,8:9].head()

,Average XPM
Timbersaw,690.000000
Medusa,676.833333
Broodmother,668.055556
Weaver,641.548387
Templar Assassin,637.923077


### Predictions related to specific players

Player with the highest kill average: Miracle-

In [26]:
players.sort_values(by='Kill Average', ascending=False, inplace=True)
players.iloc[:,1:2].head()

,Kill Average
Miracle-,11.000000
SumaiL,10.222222
Somnus丶M,9.833333
rtz,9.629630
RAMZES666,9.520000


Player with the most kills in a game: SumaiL

In [27]:
players.sort_values(by='Most Kills', ascending=False, inplace=True)
players.iloc[:,2:3].head()

,Most Kills
SumaiL,31.0
Miracle-,24.0
Somnus丶M,22.0
Sccc,22.0
Ame,21.0


Player with the highest assist average: Cr1t-

In [28]:
players.sort_values(by='Assist Average', ascending=False, inplace=True)
players.iloc[:,4:5].head()

,Assist Average
Cr1t-,19.111111
Gh,18.666667
Solo,18.640000
Fly,18.222222
fy,17.533333


Player with the lowest death average: Resolut1on

In [29]:
players.sort_values(by='Death Average', inplace=True)
players.iloc[:,3:4].head()

,Death Average
Resolut1on,3.045455
Raven,3.117647
Paparazi灬,3.150000
9pasha,3.360000
No[o]ne-,3.440000


Player with the most assists in a game: fy

In [30]:
players.sort_values(by='Most Assists', ascending=False, inplace=True)
players.iloc[:,5:6].head()

,Most Assists
fy,35.0
Solo,34.0
Cr1t-,34.0
JerAx,34.0
Fly,33.0


Player with the highest last hit average: Moonn

In [31]:
players.sort_values(by='Last Hit Average', ascending=False, inplace=True)
players.iloc[:,6:7].head()

,Last Hit Average
Moonn,341.052632
Sccc,340.736842
hFn k3 ♥ M,338.125000
Resolut1on,336.681818
Raven,334.647059


Player with the most last hits in a game: Moonn

In [32]:
players.sort_values(by='Most Last Hits', ascending=False, inplace=True)
players.iloc[:,7:8].head()

,Most Last Hits
Moonn,864.0
Raven,758.0
hFn k3 ♥ M,731.0
Ame,706.0
Topson,686.0


Player with the highest GPM in a game: Xxs

In [33]:
players.sort_values(by='Highest GPM', ascending=False, inplace=True)
players.iloc[:,9:10].head()

,Highest GPM
Xxs,1109.0
CCnC,1072.0
XinQ,1025.0
SumaiL,990.0
Miracle-,980.0


Player with the highest GPM average: Miracle-

In [34]:
players.sort_values(by='GPM Average', ascending=False, inplace=True)
players.iloc[:,8:9].head()

,GPM Average
Miracle-,625.916667
SumaiL,619.444444
Ame,605.533333
Somnus丶M,605.033333
Resolut1on,600.181818


Player with the largest hero pool: MidOne

In [35]:
players.sort_values(by='Heroes Played', ascending=False, inplace=True)
players.iloc[:,10:11].head()

,Heroes Played
MidOne,19.0
Miracle-,17.0
Fata,17.0
Ceb,17.0
Pajkatt,16.0


### Predictions related to specific teams

Team with the most kills in a game: Virtus Pro & Team Liquid (tied)

In [36]:
teams.sort_values(by='Most Kills', ascending=False, inplace=True)
teams.iloc[:,1:2].head()

,Most Kills
Virtus.pro,60.0
Team Liquid,60.0
PSG.LGD,58.0
Evil Geniuses,53.0
OG,52.0


Team with the higest kill average: Evil Geniuses

In [37]:
teams.sort_values(by='Kill Average', ascending=False, inplace=True)
teams.iloc[:,2:3].head()

,Kill Average
Evil Geniuses,34.518519
PSG.LGD,33.766667
Team Liquid,32.500000
Virtus.pro,31.160000
OG,30.000000


Team with the fewest deaths in a game: Fnatic

In [38]:
teams.sort_values(by='Fewest Deaths', inplace=True)
teams.iloc[:,3:4].head()

,Fewest Deaths
Fnatic,1.0
Team Secret,2.0
OG,6.0
Team Serenity,6.0
PSG.LGD,7.0


Team with the most assists in a game: PSG.LGD

In [39]:
teams.sort_values(by='Most Assists', ascending=False, inplace=True)
teams.iloc[:,4:5].head()

,Most Assists
PSG.LGD,128.0
INVICTUS GAMING,125.0
Mineski,121.0
OG,120.0
Optic Gaming,117.0


Team that won the longest game: Mineski

In [40]:
teams.sort_values(by='Longest Game', ascending=False, inplace=True)
teams.iloc[:,6:7].head()

,Longest Game
Mineski,[1:21:52]
Newbee,[1:19:08]
TNC Predator,[1:07:42]
OG,[1:05:21]
Evil Geniuses,[1:04:17]


Team that won the shortest game: OG

In [41]:
teams.sort_values(by='Shortest Game', inplace=True)
teams.iloc[:,7:8].head()

,Shortest Game
OG,[0:16:59]
Team Secret,[0:18:24]
Vici Gaming,[0:18:59]
VGJ Storm,[0:20:54]
Virtus.pro,[0:22:35]


Team that picked the most different heroes: Team Secret

In [42]:
teams.sort_values(by='Heroes Played', ascending=False, inplace=True)
teams.iloc[:,5:6].head()

,Heroes Played
Team Secret,58.0
PSG.LGD,53.0
Optic Gaming,52.0
Team Liquid,48.0
OG,47.0


Team that picked the least different heroes: TNC Predator

In [43]:
teams.sort_values(by='Heroes Played', inplace=True)
teams.iloc[:,5:6].head()

,Heroes Played
TNC Predator,34.0
Team Serenity,35.0
VGJ Thunder,37.0
Vici Gaming,38.0
Newbee,39.0
